# Toy Example of AttentionXML Model

## Requirements
This notebook uses the following non-standard python packages:
* numpy
* pytorch
* transformers
* treelib
* spacy
* matplotlib
* tqdm

In [1]:
import os
import spacy
import treelib
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset
from transformers import Trainer, TrainingArguments
from itertools import chain
from dataclasses import dataclass
from matplotlib import pyplot as plt
from tqdm.notebook import tqdm
from typing import Tuple, List, Set, Callable, Iterator, Iterable, Any

In [2]:
# add base directory to path
if '../' not in os.sys.path:
    os.sys.path.insert(0, '../')
# import extreme multi label stuff
from xmlc.dataset import XMLDataset
from xmlc.modules import MLP, Attention, MultiHeadAttention, PLTHierarchy
from xmlc.metrics import precision
from xmlc.tree_utils import index_tree, convert_labels_to_ids

## Paths and Hyperparameter

In [3]:
data_path = "/data/share/gsg_consulting/AttentionXML/data/ops"
fasttext_path = "/data/share/gsg_consulting/AttentionXML/models/gsg-fasttext"
output_dir = "../output/ops-fasttext-simple"

In [4]:
# data hyperparameters
max_length = 512
num_candidates = 128
# model hyperparameters
hidden_size=128
num_lstm_layers=1
dropout=0.5
# training hyperparemers
lr = 1e-3
num_epochs = 8

In [5]:
device = "cuda" if torch.cuda.is_available() else 'cpu'
print("Using device %s!" % device)

Using device cuda!


## Load raw data

In [6]:
def load_texts(fpath:str):
    with open(fpath, "r") as f:
        return f.readlines()

def load_labels(fpath:str):
    return [line.split() for line in load_texts(fpath)]

# load training data
train_texts = load_texts(os.path.join(data_path, "train_texts.txt"))
train_labels = load_labels(os.path.join(data_path, "train_labels.txt"))
assert len(train_texts) == len(train_labels)
# load test data
test_texts = load_texts(os.path.join(data_path, "test_texts.txt"))
test_labels = load_labels(os.path.join(data_path, "test_labels.txt"))
assert len(test_texts) == len(test_labels)

In [7]:
# get a list of all unique labels
unique_labels = np.unique(tuple(chain(*train_labels)))
print("# unique labels:", len(unique_labels))

# unique labels: 2703


## Build simple label tree

In [8]:
tree = treelib.Tree()
# add root node
root = tree.create_node("Root", "Root")
# no real hierarchies as all labels are in level 1
for label in unique_labels:
    tree.create_node(label, label, parent=root)

In [9]:
print("Depth:      ", tree.depth())
print("Totel nodes:", len(tree.all_nodes()))
print("Inner nodes:", len(tree.all_nodes()) - len(tree.leaves()))

Depth:       1
Totel nodes: 2704
Inner nodes: 1


In [10]:
# index the tree nodes
tree = index_tree(tree)

## Build Training and Evaluation Dataset

In [11]:
# load vocabulary
vocab = np.load(os.path.join(fasttext_path, "vocab.npy"))
embed = np.load(os.path.join(fasttext_path, "vectors.npy"))
# change special tokens
vocab[vocab == "<SEP>"] = "[SEP]"
vocab[vocab == "<PAD>"] = "[PAD]"
vocab[vocab == "<UNK>"] = "[UNK]"
# build token-id map
vocab_map = {token.lower(): i for i, token in enumerate(vocab)}

In [12]:
# get german tokenizer
from spacy.lang.de import German
# build tokenizer parameters
prefixes = German.Defaults.prefixes
suffixes = German.Defaults.suffixes
infixes = German.Defaults.infixes
prefix_search = spacy.util.compile_prefix_regex(prefixes).search if prefixes else None
suffix_search = spacy.util.compile_suffix_regex(suffixes).search if suffixes else None
infix_finditer = spacy.util.compile_infix_regex(infixes).finditer if infixes else None
# add tokenizer exception for special tokens
exc = German.Defaults.tokenizer_exceptions
exc = spacy.util.update_exc(exc, {
    '[SEP]': [{spacy.symbols.ORTH: "[SEP]"}]
})
# create tokenizer
tokenizer = spacy.tokenizer.Tokenizer(
    vocab=spacy.vocab.Vocab(strings=vocab.tolist()),
    rules=exc,
    prefix_search=prefix_search,
    suffix_search=suffix_search,
    infix_finditer=infix_finditer,
    token_match=German.Defaults.token_match,
    url_match=German.Defaults.url_match
)

In [13]:
def build_input_features(texts, max_length=256):
    # tokenize all texts and convert tokens to ids
    input_ids = []
    unk_token_id = vocab_map['[unk]']
    pad_token_id = vocab_map['[pad]']
    for text in tqdm(texts, "Tokenizing"):
        # tokenize and convert to ids
        doc = tokenizer(text)
        ids = [vocab_map.get(str(t).lower(), unk_token_id) for t in doc[:max_length]]
        ids = ids + [pad_token_id] * max(max_length - len(ids), 0)
        # add to list
        input_ids.append(ids)
    # convert to tensor
    input_ids = torch.LongTensor(input_ids)
    input_mask = (input_ids) != pad_token_id
    # return features
    return input_ids, input_mask

In [14]:
# train_input_ids, train_input_mask = build_input_features(train_texts, max_length=max_length)
# test_input_ids, test_input_mask = build_input_features(test_texts, max_length=max_length)
data = torch.load("data.bin")
train_input_ids = data["train-input-ids"]
train_input_mask = data["train-input-mask"]
test_input_ids = data["test-input-ids"]
test_input_mask = data["test-input-mask"]

In [15]:
# build train and test datasets
train_data = XMLDataset(
    input_dataset=TensorDataset(train_input_ids, train_input_mask),
    labels=convert_labels_to_ids(tree, train_labels),
    num_candidates=num_candidates
)
eval_data = XMLDataset(
    input_dataset=TensorDataset(test_input_ids, test_input_mask),
    labels=convert_labels_to_ids(tree, test_labels),
    num_candidates=num_candidates
)

## Model

In [16]:
class LSTMEncoder(nn.Module):
    """ Basic LSTM Encoder """
    
    def __init__(self, 
        embed_size:int,
        hidden_size:int, 
        num_layers:int,
        vocab_size:int,
        padding_idx:int,
        emb_init:torch.FloatTensor =None,
        dropout:float =0.2
    ) -> None:
        super(LSTMEncoder, self).__init__()
        self.dropout = dropout
        # create embedding
        self.embedding = nn.Embedding(
            num_embeddings=vocab_size,
            embedding_dim=embed_size,
            padding_idx=padding_idx,
            _weight=emb_init if emb_init is not None else None
        )
        # create lstm encoder
        self.lstm = nn.LSTM(
            input_size=embed_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True
        )
        # initial hidden and cell states for lstm
        self.h0 = nn.Parameter(torch.zeros(num_layers*2, 1, hidden_size))
        self.c0 = nn.Parameter(torch.zeros(num_layers*2, 1, hidden_size))
                
    def forward(self, 
        input_ids:torch.LongTensor, 
        input_mask:torch.BoolTensor
    ) -> torch.Tensor:
        # flatten parameters
        self.lstm.flatten_parameters()
        # pass through embedding
        b, s = input_ids.size()
        x = self.embedding.forward(input_ids)
        x = F.dropout(x, p=self.dropout, training=self.training)
        # pack padded sequences
        lengths = input_mask.sum(dim=-1).cpu()
        packed_x = nn.utils.rnn.pack_padded_sequence(
            input=x, 
            lengths=lengths, 
            batch_first=True, 
            enforce_sorted=False
        )
        # apply lstm encoder
        h0 = self.h0.repeat_interleave(b, dim=1)
        c0 = self.c0.repeat_interleave(b, dim=1)
        packed_x, _ = self.lstm(packed_x, (h0, c0))
        # unpack packed sequences
        x, _ = nn.utils.rnn.pad_packed_sequence(
            sequence=packed_x, 
            batch_first=True, 
            padding_value=0,
            total_length=s
        )
        return F.dropout(x, p=self.dropout, training=self.training)

In [17]:
class Model(nn.Module):
    """ Model Combining LSTM-Encoder and a PLT-Hierarchy Classifier """
    
    def __init__(self):
        super(Model, self).__init__()
        # create encoder
        self.enc = LSTMEncoder(
            embed_size=500, 
            hidden_size=hidden_size,
            num_layers=num_lstm_layers,
            vocab_size=vocab.shape[0], 
            padding_idx=vocab_map['[pad]'], 
            emb_init=torch.from_numpy(embed).float(),
            dropout=dropout
        )
        # create hierarchy classifier
        self.plt = PLTHierarchy(
            hidden_size=2*hidden_size, 
            num_labels=len(unique_labels),
            attention=Attention(dropout=0.5),
#             attention=MultiHeadAttention(
#                 embed_dim=2*hidden_size,
#                 num_heads=16,
#                 dropout=0.5
#             ),
            classifier=MLP(2*hidden_size, 128, 1)
        )
        
    def forward(self, input_ids, input_mask, candidates, labels=None):
        # pass through encoder
        x = self.enc(input_ids, input_mask)
        # apply classifier
        logits = self.plt(x, input_mask, candidates)
        # compute loss if targets given
        if labels is not None:
            loss = F.binary_cross_entropy_with_logits(logits, labels)
            return {'loss': loss, 'logits': logits}
        # return logits only
        return {'logits': logits}

In [18]:
# create model
model = Model().to(device)
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)
    print("Using %i GPUs!" % torch.cuda.device_count())
# create optimizer
optim = torch.optim.Adam(model.parameters(), lr=lr)

/home/IAIS/ndoll/miniconda3/envs/gsg/lib/python3.8/site-packages/torch/nn/modules/rnn.py:58: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  warnings.warn("dropout option adds dropout after all but last "


Using 2 GPUs!


In [19]:
n_trainable_params = sum((p.numel() for p in model.parameters() if p.requires_grad))
print("#Trainable Parameters: %i" % n_trainable_params)

#Trainable Parameters: 158576625


## Training
I'm way too lazy to write this from scratch so lets just use the transformers Trainer class.

In [20]:
def compute_metrics(eval_preds):
    # unpack predictions and labels
    preds, labels = eval_preds
    preds = torch.FloatTensor(preds)
    labels = torch.LongTensor(labels)
    _, preds = torch.topk(preds, k=100, dim=-1)
    # compute metrics
    return {
        "P@1": precision(preds, labels, k=1),
        "P@2": precision(preds, labels, k=2),
        "P@3": precision(preds, labels, k=3),
        "P@5": precision(preds, labels, k=5)
    }

def collate(batch):
    """ default collate and return as dictionary """
    return dict(zip(
        ('input_ids', 'input_mask', 'candidates', 'labels'),
        torch.utils.data._utils.collate.default_collate(batch)
    ))

In [21]:
# training arguments
training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    num_train_epochs=num_epochs,
    per_device_train_batch_size=128,
    per_device_eval_batch_size=256,
    save_steps=5_000,
    save_total_limit=2,
    report_to="none",
    logging_steps=250,
    eval_steps=250,
    evaluation_strategy='steps'
)
# trainer
trainer = Trainer(
    optimizers=(optim, None),
    model=model,
    args=training_args,
    data_collator=collate,
    train_dataset=train_data,
    eval_dataset=eval_data,
    compute_metrics=compute_metrics
)

In [22]:
# train the model
trainer.train()

***** Running training *****
  Num examples = 284607
  Num Epochs = 8
  Instantaneous batch size per device = 128
  Total train batch size (w. parallel, distributed & accumulation) = 256
  Gradient Accumulation steps = 1
  Total optimization steps = 8896
/home/IAIS/ndoll/miniconda3/envs/gsg/lib/python3.8/site-packages/torch/nn/parallel/_functions.py:64: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,P@1,P@2,P@3,P@5
250,0.131400,0.069999,0.469755,0.396032,0.333112,0.243497
500,0.066000,0.060807,0.515847,0.420314,0.353132,0.260090
750,0.059800,0.056927,0.554669,0.445783,0.371410,0.271637
1000,0.055700,0.052541,0.592350,0.472559,0.391890,0.288382
1250,0.051300,0.047485,0.615253,0.491518,0.407159,0.298047
1500,0.046100,0.042614,0.652316,0.519363,0.429477,0.311133
1750,0.042100,0.040377,0.693894,0.548919,0.449782,0.323203
2000,0.038600,0.035557,0.723687,0.571870,0.464845,0.331813
2250,0.035600,0.032872,0.751580,0.590140,0.478958,0.338988
2500,0.032600,0.030574,0.770872,0.607365,0.490045,0.344433


***** Running Evaluation *****
  Num examples = 21045
  Batch size = 512
***** Running Evaluation *****
  Num examples = 21045
  Batch size = 512
***** Running Evaluation *****
  Num examples = 21045
  Batch size = 512
***** Running Evaluation *****
  Num examples = 21045
  Batch size = 512
***** Running Evaluation *****
  Num examples = 21045
  Batch size = 512
***** Running Evaluation *****
  Num examples = 21045
  Batch size = 512
***** Running Evaluation *****
  Num examples = 21045
  Batch size = 512
***** Running Evaluation *****
  Num examples = 21045
  Batch size = 512
***** Running Evaluation *****
  Num examples = 21045
  Batch size = 512
***** Running Evaluation *****
  Num examples = 21045
  Batch size = 512
***** Running Evaluation *****
  Num examples = 21045
  Batch size = 512
***** Running Evaluation *****
  Num examples = 21045
  Batch size = 512
***** Running Evaluation *****
  Num examples = 21045
  Batch size = 512
***** Running Evaluation *****
  Num examples = 210

TrainOutput(global_step=8896, training_loss=0.030857581993658765, metrics={'train_runtime': 3580.265, 'train_samples_per_second': 635.946, 'train_steps_per_second': 2.485, 'total_flos': 0.0, 'train_loss': 0.030857581993658765, 'epoch': 8.0})

In [23]:
# save final model state dict to disk
torch.save(model.state_dict(), os.path.join(output_dir, "final-model.bin"))

## Plot Metrics

In [24]:
# select only train and test logs
train_logs = [log for log in trainer.state.log_history if 'loss' in log]
eval_logs = [log for log in trainer.state.log_history if 'eval_loss' in log]
# gather values from logs
train_metrics = {
    key: [log[key] for log in train_logs]
    for key in ['step', 'loss']
}
eval_logs = {
    key: [log[key] for log in eval_logs]
    for key in [
        'step', 
        'eval_loss', 
        'eval_P@1', 
        'eval_P@2', 
        'eval_P@3', 
        'eval_P@5'
    ]
}

In [25]:
fig, (ax_loss, ax_p) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
# plot losses
ax_loss.plot(train_metrics['step'], train_metrics['loss'], label="train")
ax_loss.plot(eval_logs['step'], eval_logs['eval_loss'], label="eval")
ax_loss.set(
    title="Train and Test Loss",
    ylabel="Loss"
)
ax_loss.legend()
ax_loss.grid()
# plot f1-scores
ax_p.plot(eval_logs['step'], eval_logs['eval_P@1'], label="$k=1$")
ax_p.plot(eval_logs['step'], eval_logs['eval_P@2'], label="$k=2$")
ax_p.plot(eval_logs['step'], eval_logs['eval_P@3'], label="$k=3$")
ax_p.plot(eval_logs['step'], eval_logs['eval_P@5'], label="$k=5$")
ax_p.set(
    title="Precision@k",
    xlabel="Global Step",
    ylabel="Precision"
)
ax_p.legend()
ax_p.grid()
# save figure
fig.savefig(os.path.join(output_dir, "metrics.pdf"))
plt.show()

SyntaxError: invalid syntax (507289498.py, line 24)